In [1]:
import os
os.chdir('../')

# 使用MMRazor对ResNet34进行剪枝

## 回顾MMCls

In [2]:
# Prepare config path
work_dir='./prune_example/'
config_path=f"{work_dir}/configs/"
! mkdir -p $config_path

pretrained_path='https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'

In [3]:
! echo "_base_ = ['mmcls::resnet/resnet34_8xb32_in1k.py']" > $config_path/pretrain.py
! cat $config_path/pretrain.py

# Run config
# ! python ./tools/train.py $config_path/pretrain.py

_base_ = ['mmcls::resnet/resnet34_8xb32_in1k.py']


## 准备剪枝结构搜索config
1. 生成搜索config
2. 运行搜索config
3. 获得搜索出的最优结构

### 1.生成搜索config
我们提供了一个工具可以帮助我们一条命令行生成搜索算法config。改工具需要我们提供两个参数：config路径与checkpoint路径

In [4]:
! python tools/get_search_config.py -h

usage: get_search_config.py [-h] [--flops-min FLOPS_MIN]
                            [--flops-max FLOPS_MAX] [-o O]
                            config checkpoint

positional arguments:
  config
  checkpoint

optional arguments:
  -h, --help            show this help message and exit
  --flops-min FLOPS_MIN
  --flops-max FLOPS_MAX
  -o O


In [5]:
! python tools/get_search_config.py  $config_path/pretrain.py $pretrained_path -o $config_path/search.py   &> /dev/null 
! cat $config_path/search.py 

model = dict(
    _scope_='mmrazor',
    type='SearchWrapper',
    architecture=dict(
        type='ImageClassifier',
        backbone=dict(
            type='ResNet',
            depth=34,
            num_stages=4,
            out_indices=(3, ),
            style='pytorch'),
        neck=dict(type='GlobalAveragePooling'),
        head=dict(
            type='LinearClsHead',
            num_classes=1000,
            in_channels=512,
            loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
            topk=(1, 5)),
        _scope_='mmcls',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'
        ),
        data_preprocessor=dict(
            mean=[123.675, 116.28, 103.53],
            std=[58.395, 57.12, 57.375],
            to_rgb=True)),
    mutator_cfg=dict(
        type='ChannelMutator',
        channel_unit_cfg=dict(
            type

### 2. 我们可以直接运行该config从而进行剪枝结构的搜索

In [6]:
! python ./tools/train.py $config_path/search.py  --work-dir $work_dir/search/

11/08 11:27:18 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.13 (main, Aug 25 2022, 23:26:10) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 290106469
    GPU 0: NVIDIA GeForce GTX 1660 Ti
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.3, V11.3.58
    GCC: gcc (Ubuntu 11.2.0-19ubuntu1) 11.2.0
    PyTorch: 1.12.1+cu113
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_

### 3. 获取搜索结果

In [7]:
!  cat $work_dir/search/best_fix_subnet.json

{
    "backbone.conv1_(0, 64)_64": 1.0,
    "backbone.layer1.0.conv1_(0, 64)_64": 0.19867759913483857,
    "backbone.layer1.1.conv1_(0, 64)_64": 0.6682791970899116,
    "backbone.layer1.2.conv1_(0, 64)_64": 0.5960327974045156,
    "backbone.layer2.0.conv1_(0, 128)_128": 0.32510879858428127,
    "backbone.layer2.0.conv2_(0, 128)_128": 0.9301723959494714,
    "backbone.layer2.1.conv1_(0, 128)_128": 0.07224639968539584,
    "backbone.layer2.2.conv1_(0, 128)_128": 0.8669567962247501,
    "backbone.layer2.3.conv1_(0, 128)_128": 0.2709239988202344,
    "backbone.layer3.0.conv1_(0, 256)_256": 0.4741169979354102,
    "backbone.layer3.0.conv2_(0, 256)_256": 1.0,
    "backbone.layer3.1.conv1_(0, 256)_256": 0.5553941975814806,
    "backbone.layer3.2.conv1_(0, 256)_256": 0.01,
    "backbone.layer3.3.conv1_(0, 256)_256": 0.4741169979354102,
    "backbone.layer3.4.conv1_(0, 256)_256": 0.772133396637668,
    "backbone.layer3.5.conv1_(0, 256)_256": 0.2799547987809089,
    "backbone.layer4.0.conv1_(0, 

## 准备剪枝Config

In [8]:
! python ./tools/get_prune_config.py $config_path/pretrain.py $pretrained_path -o $config_path/prune.py &> /dev/null
! cat $config_path/prune.py

model = dict(
    _scope_='mmrazor',
    type='ItePruneAlgorithm',
    architecture=dict(
        type='ImageClassifier',
        backbone=dict(
            type='ResNet',
            depth=34,
            num_stages=4,
            out_indices=(3, ),
            style='pytorch'),
        neck=dict(type='GlobalAveragePooling'),
        head=dict(
            type='LinearClsHead',
            num_classes=1000,
            in_channels=512,
            loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
            topk=(1, 5)),
        _scope_='mmcls',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'
        ),
        data_preprocessor=dict(
            mean=[123.675, 116.28, 103.53],
            std=[58.395, 57.12, 57.375],
            to_rgb=True)),
    target_pruning_ratio=dict({
        'backbone.conv1_(0, 64)_64': 1.0,
        'backbone.laye

In [9]:
# change prune config
from mmengine import fileio,Config
pruning_subnet=fileio.load(f'{work_dir}/search/best_fix_subnet.json')
pruning_config=Config.fromfile(f'{config_path}/prune.py')
pruning_config['model']['target_pruning_ratio']=pruning_subnet
pruning_config.dump(f'{config_path}/prune.py')
! cat $config_path/prune.py

model = dict(
    _scope_='mmrazor',
    type='ItePruneAlgorithm',
    architecture=dict(
        type='ImageClassifier',
        backbone=dict(
            type='ResNet',
            depth=34,
            num_stages=4,
            out_indices=(3, ),
            style='pytorch'),
        neck=dict(type='GlobalAveragePooling'),
        head=dict(
            type='LinearClsHead',
            num_classes=1000,
            in_channels=512,
            loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
            topk=(1, 5)),
        _scope_='mmcls',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'
        ),
        data_preprocessor=dict(
            mean=[123.675, 116.28, 103.53],
            std=[58.395, 57.12, 57.375],
            to_rgb=True)),
    target_pruning_ratio=dict({
        'backbone.conv1_(0, 64)_64':
        1.0,
        'backb

In [10]:
# run your prune config
! python ./tools/train.py $config_path/prune.py --work-dir $work_dir/prune

11/08 11:28:15 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.13 (main, Aug 25 2022, 23:26:10) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1856099330
    GPU 0: NVIDIA GeForce GTX 1660 Ti
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.3, V11.3.58
    GCC: gcc (Ubuntu 11.2.0-19ubuntu1) 11.2.0
    PyTorch: 1.12.1+cu113
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm

In [11]:
# ! rm -r $work_dir